## Deploy/Update endpoint

In this section we will be deploying the latest approved model to an endpoint

<b>Note: Make sure you have already approved a model in the model registry</b>


In [ ]:
import boto3
import json
import sagemaker

sess = boto3.Session()
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

In [ ]:
model_package_group_name = f"HuggingFaceSentimentPackageGroupName"
endpoint_name = "DEMO-endpoint-sentiment-analysis"

In [ ]:
from scripts.utils import get_approved_package

sm_client = boto3.client("sagemaker")

pck = get_approved_package(
    model_package_group_name
)  
model_description = sm_client.describe_model_package(ModelPackageName=pck["ModelPackageArn"])

model_description

In [ ]:
from scripts.utils import deploy_or_update_endpoint

model_package_arn = model_description["ModelPackageArn"]

deploy_or_update_endpoint(endpoint_name, model_package_arn, role, sagemaker_session)

## Test the model
Run the below to test that your model was deployed successfully and can ge

In [ ]:
%%time


runtime = boto3.client(service_name="sagemaker-runtime")

sentiment_input = {"inputs": "I love using SageMaker endpoints to serve my real-time models."}

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(sentiment_input),
    ContentType="application/json",
)

print(response["Body"].read())

In [ ]:
%%time 
for i in range(100):
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(sentiment_input),
        ContentType="application/json",
    )

In [ ]:
# # Delete the endpoint when no longer needed
# sm_client.delete_endpoint(
#     EndpointName=endpoint_name
# )